In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xw
import re
import warnings
warnings.filterwarnings('ignore')
import openpyxl as op
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
import os
import glob
import calendar
from datetime import date, timedelta, datetime

In [2]:
Date_n_ob_no_change = "32"
Date_n = Date_n_ob_no_change
current_year = "2023"
current_month = "Jul"
Month_f = current_month
Month_n = "07"
current_date = [f"{current_month}-{Date_n}-{current_year}"]

In [3]:
#Data_Not_Available_Date = ['32-Jun-2023']   #If the data is not available for more then one day then use the this code 

In [4]:
today_date = date.today().day
today_month = date.today().month
today_month_abbr = calendar.month_abbr[today_month]
today_year = date.today().year
Data_Not_Available_Date = [f"{today_date-1}-{today_month_abbr}-{current_year}"]

In [5]:
input_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout Remarks/Target Consolidated/"
output_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout Remarks/Result/"
input_loc_ob = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Is in OB?/Target/latest ob/{current_year}/{current_month}/"
input_loc_form = f"/home/dhrubakumardolai/Desktop/Python Test/All Sql Ingestion Script/Google + RWM Revisit + Visit/Result/RWM/{current_year}/{Month_f}/"
input_loc_form_2 = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Task File Rahul Sir/Form Data Append/Target/"
input_loc_revisit_con = f"/home/dhrubakumardolai/Desktop/Python Test/Result/Revisit File/{current_year}/{current_month}/Form/"
input_loc_ob_google = f"/home/dhrubakumardolai/Desktop/Python Test/Revisit File/Google Revisit/{current_year}/{Month_n}/"
input_loc_tv = f'/home/dhrubakumardolai/Desktop/Python Test/Target/Mini/Latest Team View File/{current_year}/{current_month}/'
input_loc_mini = "/home/dhrubakumardolai/Desktop/Python Test/Target/Mini/"
input_loc_tv = f"/home/dhrubakumardolai/Desktop/Python Test/Target/Mini/Latest Team View File/{current_year}/{current_month}/"
input_loc_ob_rwm = f"/home/dhrubakumardolai/Desktop/Python Test/Revisit File/OB/{current_year}/{Month_f}/{Date_n}/"
output_loc = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Payout Remarks/Result/{current_year}/{current_month}/"

In [6]:
def fill_task(row2):
    if row2['Is in Google Daily File?'] == "Yes" and row2['CompletedSubTaskTypes'] is np.nan:
        return 0
    else:
        return row2['Task Count']

In [7]:
def fill_ob_csv_file(row2, col_name):
    if row2["Validity_ob"] in ["INVALID","VALID"]:
        return "Yes"
    else:
        return row2[col_name]

In [8]:
def drop_rename(row3, old_name, new_name):
    row3 = row3.drop([old_name], axis=1)
    row3 = row3.rename(columns={new_name:old_name})
    return row3

In [9]:
def left_join_date_wise(left_table, left_date, right_table, right_date,
 common_col, merge_table_name, new_column_name, Date):
    merge_table_name = pd.DataFrame()
    for i in Date:
        t = left_table[left_table[left_date].isin([i])]
        m = right_table[right_table[right_date].isin([i])]
        t_merge = t.merge(m, on = common_col, how="left", indicator=True)
        t_merge.drop_duplicates(subset=['Primary_key'], inplace=True)
        t_merge.rename(columns={'_merge':new_column_name}, inplace=True)
        t_merge[new_column_name].replace('left_only', "No", inplace=True)
        t_merge[new_column_name].replace('both', "Yes", inplace=True)
        merge_table_name = merge_table_name.append(t_merge)
    return merge_table_name

In [10]:
def left_join_date_wise_mid(left_table, left_date, right_table, right_date,
 common_col, merge_table_name, new_column_name, Date):
    merge_table_name = pd.DataFrame()
    for i in Date:
        t = left_table[left_table[left_date].isin([i])]
        m = right_table[~right_table[right_date].isin([i])]
        t_merge = t.merge(m, on = common_col, how="left", indicator=True)
        t_merge.drop_duplicates(subset=['sno'], inplace=True)
        t_merge.rename(columns={'_merge':new_column_name}, inplace=True)
        t_merge[new_column_name].replace('left_only', "No", inplace=True)
        t_merge[new_column_name].replace('both', "Yes", inplace=True)
        merge_table_name = merge_table_name.append(t_merge)
    return merge_table_name

In [11]:
def check_source_mid(row4, source_mid_col, mexternalid_col):
    if row4[source_mid_col] is np.nan:
        return np.nan
    elif row4[source_mid_col] == row4[mexternalid_col]:
        return "TRUE"
    else:
        return "FALSE"

In [12]:
def left_join_indicator(left_table, right_table, common_col, indicator_True_False, 
left_only, both, new_column_name):
    left_table_merge = left_table.merge(right_table, how='left',on = common_col, indicator=indicator_True_False)
    left_table_merge.drop_duplicates(subset=['Primary_key'], inplace=True)
    left_table_merge.rename(columns={"_merge":new_column_name}, inplace=True)
    left_table_merge[new_column_name].replace('left_only', left_only, inplace=True)
    left_table_merge[new_column_name].replace('both', both, inplace=True)
    return left_table_merge

In [13]:
def Already_Paid(row5, Previous_Match_No_Match_col, Previous_Date_col, Ingestion_Date_col):
    if row5[Previous_Match_No_Match_col] == "No Match":
        return np.nan
    elif row5[Previous_Match_No_Match_col] == "Match" and row5[Ingestion_Date_col] == np.NaN:
        return np.nan
    elif pd.to_datetime(row5[Previous_Date_col]) >= pd.to_datetime(row5[Ingestion_Date_col]):
        return "Already Paid"
    else:
        return np.nan


# Google File

In [14]:
filelist = glob.glob(input_loc_revisit_con+"/Google/Consolidated/*.csv", recursive=True)

In [15]:
Google_file = pd.DataFrame()
for i in filelist:
    df = pd.read_csv(i)
    Google_file = Google_file.append(df)


In [16]:
Google_file['TaskSubmissionTimestamp'] = pd.to_datetime(Google_file['TaskSubmissionTimestamp'])
Google_file['TaskSubmissionTimestamp'] = Google_file['TaskSubmissionTimestamp'].dt.strftime("%d-%b-%Y")

In [17]:
Google_file['Primary_key'] = range(1,(len(Google_file)+1))

In [18]:
Google_file.head(1)

,StoreName,City,Pincode,StorePhoneNumber,OnboardingTimestamp,TaskSubmissionTimestamp,MerchantExternalId,CompletedSubTaskTypes,AdditionalQRs,AxisSTRCount,...,Dynamic IngestionStatus,Dynamic Ingestion Date,Source_MID,sno,Data Tag,Date,Report_Date,Revisit_Date,Dup,Primary_key
0,Rajeswari General Store,Kadapa,516001,9849970176,5/27/2022 16:24:43,01-Jul-2023,BCR2DN4T2C22DCZU,"VERIFY_MERCHANT, PROVISION_VPA, CAPTURE_ADDITI...",NaN,1.0,...,ADD,2023-02-03,NaN,100001,Google,01-07-2023,NaN,NaN,NaN,1


# RWM File

In [19]:
filelist = glob.glob(input_loc_revisit_con+"/RWM/Consolidated/*.csv", recursive=True)

In [20]:
RWM_file = pd.DataFrame()
for i in filelist:
    df = pd.read_csv(i)
    RWM_file = RWM_file.append(df)

In [21]:
RWM_file['Revisit Date'] = pd.to_datetime(RWM_file['Revisit Date'])
RWM_file['Revisit Date'] = RWM_file['Revisit Date'].dt.strftime("%d-%b-%Y")
Date_for_mapping = RWM_file['Revisit Date'].unique()

In [22]:
RWM_file['Primary_key'] = range(1,(len(RWM_file)+1))

In [23]:
RWM_file.head(1)

,City Name,Merchant ID,Merchant Name,Visit Count,Business Category,Owner Mobile No.,Follow Up Date,PIN Code,On Boarding Date,On Boarding Time,...,Source_MID,Hustle Task Completion Image,Data Tag,Unnamed: 33,sno,Dup_MerchantID,Report_Date,Revisit_Date,Dup,Primary_key
0,Bardhaman,9518165,KALIMATA MOBILE SERVICE CENTER,2.0,Service,9641025856,NaN,713502.0,Jun-27-2023,10:49:01,...,NaN,http://imagev1.fleetrobo.com/App_Images/FSTSIm...,RWM,NaN,100001,NaN,NaN,NaN,NaN,1


# OB_Invalid Mapping for Date and Validity

In [24]:
filelist = glob.glob(input_loc_ob+"/*.csv", recursive=True)

In [25]:
ob_con = pd.DataFrame()
for i in filelist:
    df = pd.read_csv(i)
    ob_con = ob_con.append(df)

In [26]:
ob_con['VerificationTimestamp'] = pd.to_datetime(ob_con['VerificationTimestamp'])
ob_con['VerificationTimestamp'] = ob_con['VerificationTimestamp'].dt.strftime('%d-%b-%Y')

In [27]:
ob_con_f = ob_con[ob_con['VerificationTimestamp'].str.contains(f"{current_month}")]

In [28]:
ob_con_f2 = ob_con_f[['MerchantExternalId', 'VerificationTimestamp', 'Validity']].rename(
    columns={'Validity':'Validity_ob'})

In [29]:
RWM_file.rename(columns={'Merchant Ext ID':'MerchantExternalId'}, inplace=True)

In [30]:
RWM_file_2 = pd.DataFrame()
RWM_file_2 = left_join_date_wise(RWM_file, 'Revisit Date', ob_con_f2, 'VerificationTimestamp',
 'MerchantExternalId', RWM_file_2, 'Is_in_OB_CSV_file', Date_for_mapping)

# OB_Google_Invalid Mapping for Date and Validity

In [31]:
filelist = glob.glob(input_loc_ob_google+"**/*/onboardings/data-00000-of-00001.csv", recursive = True)

In [32]:
google_csv_ob = pd.DataFrame()

for file in filelist:
    if file.endswith(".csv"):
        df = pd.read_csv(file)
        google_csv_ob = google_csv_ob.append(df)

In [33]:
google_csv_ob['VerificationTimestamp'] = pd.to_datetime(google_csv_ob['VerificationTimestamp'])
google_csv_ob['VerificationTimestamp'] = google_csv_ob['VerificationTimestamp'].dt.strftime('%d-%b-%Y')

In [34]:
google_csv_ob_f = google_csv_ob[google_csv_ob['VerificationTimestamp'].str.contains(f"{current_month}")]

In [35]:
google_csv_ob_f2 = google_csv_ob_f[['MerchantExternalId', 'VerificationTimestamp', 'Validity']].rename(
    columns={'Validity':'Validity_google_ob'})

In [36]:
RWM_file_3 = pd.DataFrame()
RWM_file_3 = left_join_date_wise(RWM_file_2, 'Revisit Date', google_csv_ob_f2, 'VerificationTimestamp',
 'MerchantExternalId', RWM_file_3, 'Is_in_OB_Google_CSV_file', Date_for_mapping)

In [37]:
RWM_file_3['VerificationTimestamp_x'].fillna(RWM_file_3['VerificationTimestamp_y'], inplace=True)
RWM_file_3['Is_in_OB_CSV_file'].replace("No", "", inplace=True)
RWM_file_3['Is_in_OB_CSV_file'] = RWM_file_3['Is_in_OB_CSV_file'].astype('str')
RWM_file_3['Is_in_OB_CSV_file'].fillna(RWM_file_3['Is_in_OB_Google_CSV_file'], inplace=True)
RWM_file_3['Validity_ob'].fillna(RWM_file_3['Validity_google_ob'], inplace=True)
RWM_file_3.drop(['VerificationTimestamp_y', 'Is_in_OB_Google_CSV_file', 
'Validity_google_ob'], axis=1, inplace=True)

In [38]:
RWM_file_3.rename(columns={'VerificationTimestamp_x':'OB_Date'}, inplace=True)

In [39]:
RWM_file_3['Validity_ob'] = RWM_file_3['Validity_ob'].str.replace(" ", "", regex=True)

In [40]:
RWM_file_3['Validity_ob_2'] = RWM_file_3.apply(fill_ob_csv_file, axis=1, args=("Validity_ob",))

In [41]:
RWM_file_4 = drop_rename(RWM_file_3, 'Is_in_OB_CSV_file', 'Validity_ob_2')

In [42]:
RWM_file_4['Source MID Match Check'] = RWM_file_4.apply(check_source_mid, args=("Source_MID", 
"MerchantExternalId"), axis=1)

# RWM OB Date and Validity Remarks

In [43]:
rwm_ob_validity = pd.read_csv(input_loc_ob_rwm+"Is Suspicious_Updated.csv")

In [44]:
rwm_ob_validity.head(1)

,Merchant ID,Is Suspicious,Final
0,9531548,No,Valid


In [45]:
rwm_ob_validity_f = rwm_ob_validity[['Merchant ID', 'Final']]

In [46]:
External_Gpay_fos = pd.read_csv(input_loc_ob_rwm+"External Gpay FOS Backend_Trail - Gpay Internal.csv")

In [47]:
External_Gpay_fos_f = External_Gpay_fos[['On Boarding Date', 'Owner Mob No', 'Merchant ID']]

In [48]:
External_Gpay_fos_f['On Boarding Date'] = pd.to_datetime(External_Gpay_fos_f['On Boarding Date'])
External_Gpay_fos_f['On Boarding Date'] = External_Gpay_fos_f['On Boarding Date'].dt.strftime("%d-%b-%Y")

In [49]:
External_Gpay_fos_f_merge = External_Gpay_fos_f.merge(rwm_ob_validity_f, on="Merchant ID", how="left")

In [50]:
External_Gpay_fos_f_merge.head(1)

,On Boarding Date,Owner Mob No,Merchant ID,Final
0,01-Jul-2023,BCR2DN4T53A4XX2V,9531548,Valid


In [51]:
External_Gpay_fos_f_merge_f = External_Gpay_fos_f_merge[['Owner Mob No', 'Final', 'On Boarding Date']].rename(
    columns={'On Boarding Date':'RWM OB Date', 'Owner Mob No':'MerchantExternalId'})

In [52]:
len(External_Gpay_fos_f_merge) - len(External_Gpay_fos_f_merge['Owner Mob No'])

0

# Already Paid with Google

In [53]:
already_paid = pd.read_csv(input_loc_mini+"Previous Valid.csv")
already_paid_2 = pd.read_csv(input_loc_mini+"Previous Valid 2.csv")

In [54]:
already_paid['Date'] = pd.to_datetime(already_paid['TaskSubmissionTimestamp'])
already_paid = already_paid[~already_paid['MerchantExternalId'].isin([np.nan])]
already_paid['Date'].fillna(method='ffill', inplace=True)
already_paid['Date'].fillna(method='bfill', inplace=True)
already_paid['Date'] = already_paid['Date'].dt.strftime("%Y-%m-%d")

In [55]:
already_paid.head(1)

,TaskSubmissionTimestamp,MerchantExternalId,Date,Validity
0,07/06/22 11:26,BCR2DN4T2CA2D3JX,2022-07-06,VALID


In [56]:
already_paid_2['Date'] = pd.to_datetime(already_paid_2['TaskSubmissionTimestamp'])
already_paid_2 = already_paid_2[~already_paid_2['MerchantExternalId'].isin([np.nan])]
already_paid_2['Date'].fillna(method='ffill', inplace=True)
already_paid_2['Date'].fillna(method='bfill', inplace=True)
already_paid_2['Date'] = already_paid_2['Date'].dt.strftime("%Y-%m-%d")

In [ ]:
already_paid_complete = already_paid.append(already_paid_2)

In [ ]:
already_paid_complete_f = already_paid_complete.drop_duplicates(subset=['MerchantExternalId'], keep="last")

In [ ]:
already_paid_complete_f2 = already_paid_complete_f[['MerchantExternalId', 'Date']]

In [ ]:
already_paid_complete_f2.head(1)

,MerchantExternalId,Date
0,BCR2DN4T2CA2D3JX,2022-07-06


In [ ]:
already_paid_complete_f2.rename(columns={'Date':'Is_in_Previous_Month_Payout_Date'}, inplace=True)

In [ ]:
RWM_file_5 = left_join_indicator(RWM_file_4, already_paid_complete_f2, 'MerchantExternalId', 
True, "No Match", "Match", "Is_in_Previous_Month_Payout")

In [ ]:
RWM_file_5['Already_Paid'] = RWM_file_5.apply(Already_Paid, args=("Is_in_Previous_Month_Payout", "Is_in_Previous_Month_Payout_Date", 'Report Date'), axis=1)

# OB_Date_Check for RWM

In [ ]:
RWM_file_5['OB_Date_Check'] = RWM_file_5.apply(check_source_mid, args=("OB_Date", 
"Revisit Date"), axis=1)

# Is_in_Google_Daily_File_

In [ ]:
Google_file_f_Date = Google_file[['MerchantExternalId', 'TaskSubmissionTimestamp']]

In [ ]:
RWM_file_6 = pd.DataFrame()
RWM_file_6 = left_join_date_wise(RWM_file_5, 'Revisit Date', Google_file_f_Date, 'TaskSubmissionTimestamp',
 'MerchantExternalId', RWM_file_6, 'Is_in_Google_Daily_File_2', Date_for_mapping)

In [ ]:
RWM_file_6.drop(['TaskSubmissionTimestamp'], axis=1, inplace=True)

In [ ]:
RWM_file_6.drop(['PQR_Axis.1', 'PQR_Icici.1', 'Unnamed: 33', 'Unnamed: 33'],axis=1, inplace=True)

In [ ]:
RWM_file_6['MerchantExternalId'].replace("Empty", np.nan, inplace=True)

In [ ]:
RWM_file_6['Revisit Date'] = pd.to_datetime(RWM_file_6['Revisit Date'])
RWM_file_6.sort_values(by=['Validity','MerchantExternalId','Revisit Date'], ascending=[False, True, True], inplace=True)
RWM_file_6['Revisit Date'] = RWM_file_6['Revisit Date'].dt.strftime("%d-%b-%Y")

In [ ]:
RWM_file_6['Ext_ID Dup?'] = RWM_file_6.groupby("MerchantExternalId")["MerchantExternalId"].rank(method='first')
RWM_file_6['Ext_ID Dup?'].replace(np.nan, 0, inplace=True)
RWM_file_6['Merchant_ID_Dup'] = RWM_file_6.groupby("Merchant ID")["Merchant ID"].rank(method='first')
RWM_file_6['Merchant_ID_Dup'].replace(np.nan, 0, inplace=True)
RWM_file_6['Phone_Dup?'] = RWM_file_6.groupby("Owner Mobile No.")["Owner Mobile No."].rank(method='first')
RWM_file_6['Phone_Dup?'].replace(np.nan, 0, inplace=True)

In [ ]:
RWM_file_6.drop_duplicates(subset=['Merchant ID'], inplace=True)

In [ ]:
RWM_file_6['Task Count_2'] = RWM_file_6.apply(fill_task, axis=1)
RWM_file_6.drop(['Task Count'], axis=1, inplace=True)
RWM_file_6.rename(columns={'Task Count_2':'Task Count'}, inplace=True)

# RWM Valid OB with Date

In [ ]:
RWM_file_7 = pd.DataFrame()
RWM_file_7 = left_join_date_wise(RWM_file_6, 'Revisit Date', External_Gpay_fos_f_merge_f, 'RWM OB Date',
 'MerchantExternalId', RWM_file_6, 'Is_in_OB_RWM_file', Date_for_mapping)

In [ ]:
RWM_file_6 = RWM_file_7

# Payout Criteria

In [ ]:
RWM_file_6['Source Data picekd from?'] = RWM_file_6['Data Tag'].str.contains('Form', na=False)
RWM_file_6['Source Data picekd from?'] = RWM_file_6['Source Data picekd from?'].replace(False, "RWM")
RWM_file_6['Source Data picekd from?'] = RWM_file_6['Source Data picekd from?'].replace(True, "Form")
RWM_file_6['Invalid_PhonenumberLookup'] = RWM_file_6['Invalid Reason'].str.contains('Invalid_PhonenumberLookup', na=False)
RWM_file_6['Invalid_NotOnTargetlist'] = RWM_file_6['Invalid Reason'].str.contains('Invalid_NotOnTargetlist', na=False)
RWM_file_6['Invalid_IncompleteSubTasks'] = RWM_file_6['Invalid Reason'].str.contains('Invalid_IncompleteSubTasks', na=False)

# Check Data Available in Google

In [ ]:
check_date = lambda x:x in Data_Not_Available_Date
RWM_file_6['Data_Avaliable_Google_file'] = RWM_file_6['Revisit Date'].apply(check_date)

# Payout Citeria

In [ ]:
RWM_file_6.rename(columns={'Is in Source File?_x':'Is in Source File?',
'Is_in_Google_Daily_File_':'Is in Google Daily File?', 'Source MID Match Check':'Dynamic Source MID Check',
'Is_in_OB_CSV_file':'Present in Google Invalid?', 'Source MID Match Check':'Dynamic Source MID Check'}, inplace=True)

In [ ]:
RWM_file_6['Dynamic Source MID Check'].replace('FALSE', False, inplace=True)

In [ ]:
RWM_file_6['Dynamic Source MID Check'].unique()

array([nan, False], dtype=object)

In [ ]:
def payout_remarks(row):
   if row['Ext_ID Dup?'] > 1:
       return 'Duplicate Ext_ID'
   elif row['Ext_ID Dup?'] == 0:
       return 'Not Found'
   elif row['Phone_Dup?'] > 1:
        return "Duplicate Phone Number"
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] == "Yes" and row['Dynamic Source MID Check'] == False:
       return 'Possible -Re OB'
       
   elif row['Data_Avaliable_Google_file'] == True:
        if row['Is in Source File?'] == "Match" or row['Is in Source File?'] == "MN Match":
            return "Valid"
        else:
            return "Not Found, Invalid_NotOnTargetlist"
   elif row['Final'] == "Valid":
       return 'Considered in Onboarding'
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] == "Yes" and row['Is in Source File?'] == 'No Match':
       return 'Not Found, Invalid_NotOnTargetlist'
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] != "Yes" and row['Dynamic Source MID Check'] != False:
       if row['Is in Source File?'] == 'Match':
           return 'Not Found'
       else:
           return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Present in Google Invalid?'] == np.nan and row['Is in Source File?'] in ('No Match','No Match Dynamic'):
       return 'Not Found, Invalid_NotOnTargetlist'
   elif row['Is in Google Daily File?'] == "No" and row['Source Data picekd from?'] == 'Form Data' and row['MerchantExternalId'] == np.nan:
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Source Data picekd from?'] == 'RWM' and row['MerchantExternalId'] == np.nan:
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Source Data picekd from?'] == 'RWM' and row['Is in Source File?'] == 'Match':
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "No" and row['Source Data picekd from?'] == 'RWM' and row['Is in Source File?'] == 'MN Match':
       return 'Not Found'
   elif row['Is in Google Daily File?'] == "Yes" : 
       if row['Task Count'] < 3:
           if row['Invalid_PhonenumberLookup'] == True:
               return 'Invalid_PhonenumberLookup'
           else:
               return 'Invalid_IncompleteSubTasks'
       elif row['Task Count'] >= 3:
           if row['Is in Source File?'] == 'MN Match' and row['Validity'] == "INVALID":
               return "Pending Review"
           elif row['Is in Source File?'] != 'MN Match' and row['Invalid_PhonenumberLookup'] == True:
               return "Invalid_PhonenumberLookup"
           elif row['Is in Source File?'] != 'MN Match' and row['Invalid_NotOnTargetlist'] == True:
               return "Invalid_NotOnTargetlist"
           else:
               if row['Validity'] == "VALID":
                    return "Valid"
               else:
                    return np.nan
   else:
       return "Not Found"



In [ ]:
RWM_file_6['Payout_remarks'] = RWM_file_6.apply(payout_remarks, axis=1)

In [ ]:
def Already_Paid_Col(row6, Payout_col, Already_Paid_col):
    if row6[Payout_col] in ("Invalid_PhonenumberLookup", "Invalid_NotOnTargetlist", "Invalid_IncompleteSubTasks") and row6[Already_Paid_col] == "Already Paid":
        return "Already Paid"
    else:
        return row6[Payout_col]


In [ ]:
RWM_file_6['Payout_remarks_2'] = RWM_file_6.apply(Already_Paid_Col, args=("Payout_remarks", "Already_Paid"),axis=1)
RWM_file_6.drop(['Payout_remarks'], axis=1, inplace=True)
RWM_file_6.rename(columns={'Payout_remarks_2':'Payout_remarks'}, inplace=True)

In [ ]:
RWM_file_7 = RWM_file_6[['City Name', 'Merchant ID', 'Owner Mobile No.', 'Employee Code', 'Role Name',
'POD Name', 'POD ID', 'Team Lead Name', 'Revisit Date', 'MerchantExternalId', 'Is in Google Daily File?',
'CompletedSubTaskTypes','Task Count', 'Is in Source File?', 'Validity', 'Invalid Reason', 'Segment', 'Is in Dynamic Source Visit Date Valid?',
'Report Date', 'IngestionStatus', 'Ingestion Date', 'Revisit Expire Date', 'Source_MID', 'OB_Date', 'Final', 'Present in Google Invalid?', 'Dynamic Source MID Check',
'Is_in_Previous_Month_Payout_Date', 'Is_in_Previous_Month_Payout', 'OB_Date_Check','Ext_ID Dup?',
'Merchant_ID_Dup', 'Phone_Dup?' ,'Data Tag', 'Payout_remarks']].rename(columns={"Final":'Validity_ob', 'RWM OB Date':'OB_Date'})

In [ ]:
rwm_col = RWM_file_7.columns

In [ ]:
for i in rwm_col:
    RWM_file_7[i].replace("^ ", "", inplace=True, regex=True)
    RWM_file_7[i].replace("$ ", "", inplace=True, regex=True)
    RWM_file_7[i].replace(r'\r+|\n+|\t+','', inplace=True, regex=True)

In [ ]:
RWM_file_7['Merchant ID'] = RWM_file_7['Merchant ID'].apply('int64')

In [ ]:
RWM_file_7.to_csv(output_loc+"payout_remarks_rwm.csv", index=False)

# Google Analysis

In [ ]:
Google_file.head(1)

,StoreName,City,Pincode,StorePhoneNumber,OnboardingTimestamp,TaskSubmissionTimestamp,MerchantExternalId,CompletedSubTaskTypes,AdditionalQRs,AxisSTRCount,...,Dynamic IngestionStatus,Dynamic Ingestion Date,Source_MID,sno,Data Tag,Date,Report_Date,Revisit_Date,Dup,Primary_key
0,Rajeswari General Store,Kadapa,516001,9849970176,5/27/2022 16:24:43,01-Jul-2023,BCR2DN4T2C22DCZU,"VERIFY_MERCHANT, PROVISION_VPA, CAPTURE_ADDITI...",NaN,1.0,...,ADD,2023-02-03,NaN,100001,Google,01-07-2023,NaN,NaN,NaN,1


### OB_Invalid Mapping for Date and Validity

In [ ]:
Google_file_2 = pd.DataFrame()
Google_file_2 = left_join_date_wise(Google_file, 'TaskSubmissionTimestamp', ob_con_f2, 'VerificationTimestamp',
 'MerchantExternalId', Google_file_2, 'Is_in_OB_CSV_file', Date_for_mapping)

### OB_Google_Invalid Mapping for Date and Validity

In [ ]:
Google_file_3 = pd.DataFrame()
Google_file_3 = left_join_date_wise(Google_file_2, 'TaskSubmissionTimestamp', google_csv_ob_f2, 'VerificationTimestamp',
 'MerchantExternalId', Google_file_3, 'Is_in_OB_Google_CSV_file', Date_for_mapping)

In [ ]:
Google_file_3['VerificationTimestamp_x'].fillna(Google_file_3['VerificationTimestamp_y'], inplace=True)
Google_file_3['Is_in_OB_CSV_file'].replace("No", "", inplace=True)
Google_file_3['Is_in_OB_CSV_file'] = Google_file_3['Is_in_OB_CSV_file'].astype('str')
Google_file_3['Is_in_OB_CSV_file'].fillna(Google_file_3['Is_in_OB_Google_CSV_file'], inplace=True)
Google_file_3['Validity_ob'].fillna(Google_file_3['Validity_google_ob'], inplace=True)
Google_file_3.drop(['VerificationTimestamp_y', 'Is_in_OB_Google_CSV_file', 
'Validity_google_ob'], axis=1, inplace=True)

In [ ]:
Google_file_3.rename(columns={'VerificationTimestamp_x':'OB_Date'}, inplace=True)

In [ ]:
Google_file_3['Validity_ob'] = Google_file_3['Validity_ob'].str.replace(" ", "", regex=True)

In [ ]:
Google_file_3['Validity_ob_2'] = Google_file_3.apply(fill_ob_csv_file, axis=1, args=("Validity_ob",))

In [ ]:
Google_file_4 = drop_rename(Google_file_3, 'Is_in_OB_CSV_file', 'Validity_ob_2')

In [ ]:
Google_file_4['Source MID Match Check'] = Google_file_4.apply(check_source_mid, args=("Source_MID", 
"MerchantExternalId"), axis=1)

# Already Paid with Google

In [ ]:
Google_file_5 = left_join_indicator(Google_file_4, already_paid_complete_f2, 'MerchantExternalId', 
True, "No Match", "Match", "Is_in_Previous_Month_Payout")

In [ ]:
Google_file_5['Already_Paid'] = Google_file_5.apply(Already_Paid, args=("Is_in_Previous_Month_Payout", "Is_in_Previous_Month_Payout_Date", 'Dynamic Ingestion Date'), axis=1)

# OB_Date_Check for Google

In [ ]:
Google_file_5['OB_Date_Check'] = Google_file_5.apply(check_source_mid, args=("OB_Date", 
"TaskSubmissionTimestamp"), axis=1)

# Is_in_RWM_Daily_File_

In [ ]:
RWM_file_f_Date = RWM_file[['MerchantExternalId', 'Revisit Date']]

In [ ]:
Google_file_6 = pd.DataFrame()
Google_file_6 = left_join_date_wise(Google_file_5, 'TaskSubmissionTimestamp', RWM_file_f_Date,'Revisit Date' ,
 'MerchantExternalId', Google_file_6, 'Is_in_RWM_Daily_File_2', Date_for_mapping)

In [ ]:
Google_file_6.drop(['Revisit Date'], axis=1, inplace=True)

In [ ]:
Google_file_6.drop(['Unnamed: 33'],axis=1, inplace=True)

In [ ]:
Google_file_6['MerchantExternalId'].replace("Empty", np.nan, inplace=True)

In [ ]:
Google_file_6['TaskSubmissionTimestamp'] = pd.to_datetime(Google_file_6['TaskSubmissionTimestamp'])
Google_file_6.sort_values(by=['Validity','MerchantExternalId','TaskSubmissionTimestamp'], ascending=[False, True, True], inplace=True)
Google_file_6['TaskSubmissionTimestamp'] = Google_file_6['TaskSubmissionTimestamp'].dt.strftime("%d-%b-%Y")

In [ ]:
Google_file_6['Ext_ID Dup?'] = Google_file_6.groupby("MerchantExternalId")["MerchantExternalId"].rank(method='first')
Google_file_6['Ext_ID Dup?'].replace(np.nan, 0, inplace=True)
Google_file_6['Phone_Dup?'] = Google_file_6.groupby("StorePhoneNumber")["StorePhoneNumber"].rank(method='first')
Google_file_6['Phone_Dup?'].replace(np.nan, 0, inplace=True)

# Payout Criteria

In [ ]:
Google_file_6['Source Data picekd from?'] = Google_file_6['Data Tag'].str.contains('Form', na=False)
Google_file_6['Source Data picekd from?'] = Google_file_6['Source Data picekd from?'].replace(False, "RWM")
Google_file_6['Source Data picekd from?'] = Google_file_6['Source Data picekd from?'].replace(True, "Form")
Google_file_6['Invalid_PhonenumberLookup'] = Google_file_6['InvalidReasons'].str.contains('Invalid_PhonenumberLookup', na=False)
Google_file_6['Invalid_NotOnTargetlist'] = Google_file_6['InvalidReasons'].str.contains('Invalid_NotOnTargetlist', na=False)
Google_file_6['Invalid_IncompleteSubTasks'] = Google_file_6['InvalidReasons'].str.contains('Invalid_IncompleteSubTasks', na=False)

# Payout Citeria

In [ ]:
Google_file_6.rename(columns={'Is_in_RWM_Daily_File_2':'Is in RWM Daily File?', 'Source MID Match Check':'Dynamic Source MID Check',
'Is_in_OB_CSV_file':'Present in Google Invalid?', 'Source MID Match Check':'Dynamic Source MID Check'}, inplace=True)

In [ ]:
Google_file_6['Dynamic Source MID Check'].replace('FALSE', False, inplace=True)

In [ ]:
Google_file_6['Dynamic Source MID Check'].unique()

array([nan, False], dtype=object)

In [ ]:
Google_file_6.to_csv("Test_1.csv", index=False)

In [ ]:
def payout_remarks(row):
   if row['Ext_ID Dup?'] > 1:
       return 'Duplicate Ext_ID'
   elif row['Ext_ID Dup?'] == 0:
       return 'Not Found'
   elif row['Phone_Dup?'] > 1:
        return "Duplicate Phone Number"      
   elif row['Validity_ob'] == "VALID":
       return 'Considered in Onboarding'
   elif row['Is in RWM Daily File?'] == "Yes" or row['Is in RWM Daily File?'] == "No" : 
       if row['Task Count'] < 3:
           if row['Invalid_PhonenumberLookup'] == True:
               return 'Invalid_PhonenumberLookup'
           else:
               return 'Invalid_IncompleteSubTasks'
       elif row['Task Count'] >= 3:
           if row['Is in Source File?'] == 'MN Match' and row['Validity'] == "INVALID":
               return "Pending Review"
           elif row['Is in Source File?'] != 'MN Match' and row['Invalid_PhonenumberLookup'] == True:
               return "Invalid_PhonenumberLookup"
           elif row['Is in Source File?'] != 'MN Match' and row['Invalid_NotOnTargetlist'] == True:
               return "Invalid_NotOnTargetlist"
           else:
               if row['Validity'] == "VALID":
                    return "Valid"
               else:
                    return np.nan
   else:
       return "Not Found"



In [ ]:
Google_file_6['Payout_remarks'] = Google_file_6.apply(payout_remarks, axis=1)

In [ ]:
Google_file_6['Payout_remarks_2'] = Google_file_6.apply(Already_Paid_Col, args=("Payout_remarks", "Already_Paid"),axis=1)
Google_file_6.drop(['Payout_remarks'], axis=1, inplace=True)
Google_file_6.rename(columns={'Payout_remarks_2':'Payout_remarks'}, inplace=True)

In [ ]:
Google_file_7 = Google_file_6[['City', 'Pincode','StorePhoneNumber', 'TaskSubmissionTimestamp',
'MerchantExternalId', 'CompletedSubTaskTypes', 'Validity', 'InvalidReasons', 'AgentEmail',
'Is in Source File?', 'Is in RWM Daily File?', 'IF R is Blank then RWM App Code', 'Team View City', 'Task Count',
'Is in Google Daily OB File?', 'Is Previous Revisit same month?', 'Previous Revisit Date',
'Segments', 'Is in Dynamic Source Visit Date Valid?', 'Revisit Expire Date','Dynamic IngestionStatus',
'Dynamic Ingestion Date', 'Validity_ob','OB_Date_Check', 'Present in Google Invalid?','Source_MID', 
'Dynamic Source MID Check','Is_in_Previous_Month_Payout_Date', 'Is_in_Previous_Month_Payout','Ext_ID Dup?',
'Phone_Dup?','Source Data picekd from?',"Data Tag", 'Payout_remarks']]

In [ ]:
Google_col = Google_file_7.columns

In [ ]:
for i in Google_col:
    Google_file_7[i].replace("^ ", "", inplace=True, regex=True)
    Google_file_7[i].replace("$ ", "", inplace=True, regex=True)
    Google_file_7[i].replace(r'\r+|\n+|\t+','', inplace=True, regex=True)

In [ ]:
Google_file_7.to_csv(output_loc+"Reconsilation_remarks_Google.csv", index=False)

In [ ]:
len(Google_file_7)

96588

In [ ]:
len(RWM_file_7)

97363